In [1]:
import platform,os
os_name = platform.system()
hdfs_home = "/user/" + os.getenv("HOME").split("/")[2]

In [2]:
print(hdfs_home)
print(os_name)

/user/kranthidr5051
Linux


In [3]:
path = hdfs_home+""

In [4]:
spark_home = "/usr/hdp/current/spark2-client"

In [5]:
import findspark
findspark.init(spark_home)
#findspark.init()
findspark.find()

'/usr/hdp/current/spark2-client'

In [6]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("sg12-rddBasics").getOrCreate()
#spark = SparkSession.builder.master("local[*]").appName("sg12-rddBasics").getOrCreate()

In [7]:
sc = spark.sparkContext

In [8]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://ip-172-31-35-141.ec2.internal:8088/proxy/application_1534772926501_6719


In [9]:
RDD = spark.range(10).rdd

In [10]:
RDD.take(2)

[Row(id=0), Row(id=1)]

In [11]:
# COMMAND ----------
RDD1 = spark.range(10).toDF("id").rdd.map(lambda row: row[0])

In [12]:
RDD1.take(2)

[0, 1]

In [13]:
spark.range(10).rdd.map(lambda row: row[0]).take(2)

[0, 1]

In [14]:
spark.range(10).rdd.map(lambda row: row).take(2)

[Row(id=0), Row(id=1)]

In [15]:
# COMMAND ----------
spark.range(10).rdd.toDF().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [16]:
# COMMAND ----------
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")

In [17]:
words = spark.sparkContext.parallelize(myCollection, 2)

In [18]:
# COMMAND ----------
words.setName("myWords")

myWords ParallelCollectionRDD[38] at parallelize at PythonRDD.scala:175

In [19]:
words.name() # myWords

u'myWords'

In [20]:
words.collect()

['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [21]:
# COMMAND ----------
def startsWithS(individual):
  return individual.startswith("S")

In [22]:
# COMMAND ----------
words.filter(lambda word: startsWithS(word)).collect()

['Spark', 'Simple']

In [23]:
# COMMAND ----------
words2 = words.map(lambda word: (word, word[0], word.startswith("S")))

In [24]:
words2.collect()

[('Spark', 'S', True),
 ('The', 'T', False),
 ('Definitive', 'D', False),
 ('Guide', 'G', False),
 (':', ':', False),
 ('Big', 'B', False),
 ('Data', 'D', False),
 ('Processing', 'P', False),
 ('Made', 'M', False),
 ('Simple', 'S', True)]

In [25]:
# COMMAND ----------

words2.filter(lambda record: record[2]).take(5)

[('Spark', 'S', True), ('Simple', 'S', True)]

In [26]:
# COMMAND ----------

words.flatMap(lambda word: list(word)).count()

51

In [27]:
# COMMAND ----------
words.sortBy(lambda word: len(word) * -1).take(2)

['Definitive', 'Processing']

In [28]:
# COMMAND ----------
fiftyFiftySplit = words.randomSplit([0.5, 0.5])

In [29]:
fiftyFiftySplit[0].collect()

['Spark', 'The', 'Definitive', 'Processing', 'Made', 'Simple']

In [30]:
fiftyFiftySplit[1].collect()

['Guide', ':', 'Big', 'Data']

In [31]:
# COMMAND ----------
spark.sparkContext.parallelize(range(1, 21)).reduce(lambda x, y: x + y) # 210

210

In [52]:
rddPair = words.map(lambda word: (word, float(len(word))))

In [53]:
rddPair.collect()

[('Spark', 5.0),
 ('The', 3.0),
 ('Definitive', 10.0),
 ('Guide', 5.0),
 (':', 1.0),
 ('Big', 3.0),
 ('Data', 4.0),
 ('Processing', 10.0),
 ('Made', 4.0),
 ('Simple', 6.0)]

In [54]:
rddPair.filter(lambda record: record[1]>5).take(5)

[('Definitive', 10.0), ('Processing', 10.0), ('Simple', 6.0)]

In [55]:
rddPair.map(lambda record: record[0]).collect()

['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [56]:
rddPair.values().collect()

[5.0, 3.0, 10.0, 5.0, 1.0, 3.0, 4.0, 10.0, 4.0, 6.0]

In [57]:
rddPair.map(lambda record: record[1]).collect()

[5.0, 3.0, 10.0, 5.0, 1.0, 3.0, 4.0, 10.0, 4.0, 6.0]

In [58]:
def sumWordLength(left, right):
    return left+right

rddPair.values().reduce(sumWordLength)
# TypeError: 'float' object has no attribute '__getitem__'

51.0

In [59]:
rddPair.values().reduce(lambda x, y: x+y)

#rddPair.reduce(lambda x, y: x[1]+y[1])
# TypeError: 'float' object has no attribute '__getitem__'

51.0

In [49]:
rddPair.reduce(lambda x, y: (x[0], x[1]+y[1]))

('Spark', 51.0)

In [50]:
rddPair.reduce(lambda x, y: ("sumValues", x[1]+y[1]))

('sumValues', 51.0)

In [60]:
rddPair.reduce(lambda x, y: (_, x[1]+y[1]))

(51.0, 51.0)

In [40]:
# COMMAND ----------

def wordLengthReducer(leftWord, rightWord):
  if len(leftWord) > len(rightWord):
    return leftWord
  else:
    return rightWord

In [41]:
words.reduce(wordLengthReducer)

'Processing'

In [42]:
# COMMAND ----------
words.getStorageLevel()

StorageLevel(False, False, False, False, 1)

In [43]:
# COMMAND ----------
words.mapPartitions(lambda part: [1]).sum() # 2

2

In [44]:
# COMMAND ----------
def indexedFunc(partitionIndex, withinPartIterator):
  return ["partition: {} => {}".format(partitionIndex,x) for x in withinPartIterator]

In [45]:
words.mapPartitionsWithIndex(indexedFunc).collect()

['partition: 0 => Spark',
 'partition: 0 => The',
 'partition: 0 => Definitive',
 'partition: 0 => Guide',
 'partition: 0 => :',
 'partition: 1 => Big',
 'partition: 1 => Data',
 'partition: 1 => Processing',
 'partition: 1 => Made',
 'partition: 1 => Simple']

In [46]:
spark.sparkContext.parallelize(["Hello", "World"], 2).collect()

['Hello', 'World']

In [47]:
# COMMAND ----------
spark.sparkContext.parallelize(["Hello", "World"], 2).glom().collect()
# [['Hello'], ['World']]


[['Hello'], ['World']]